In [1]:
# libraries
import os
import random
import time

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
import seaborn as sns
import sklearn
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold, KFold, RepeatedKFold
from xgboost import XGBRegressor

seed = 42
np.random.seed(seed)
random.seed(seed)
pd.set_option("display.max_columns", 50)

/home/balde/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
emp = pd.read_csv("employment_status.csv")
ss = pd.read_csv("SampleSubmission.csv")
income = pd.read_csv("income_group.csv")
custom = pd.read_csv("customer.csv")
transaction = pd.read_csv("transactions.csv")

In [3]:
train.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE,DECLARED_NET_INCOME
0,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,"13,500"
1,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,"9,000"
2,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,"8,000"
3,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,"7,000"
4,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,"10,400"


In [4]:
test.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,2022-01
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03


In [5]:
transaction.head()

,TRANSACTION_DESCRIPTION,CUSTOMER_IDENTIFIER,RECORD_DATE,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE
0,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-13 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,1857.56,1000.00,11785,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
1,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-02-26 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,8843.46,-2000.00,11348,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
2,POS PURCHASE,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,23.82,-199.98,148,CHEQ,system,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,04ae923a903fb07b5c846eb61eda25c4766800f02c2e09...
3,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-03-27 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,198.11,-79.39,11419,SAVE,system,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
4,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-05-06 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,13631.60,-200.00,11528,SAVE,atm,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN


In [6]:
transaction.shape, train.shape

((847544, 11), (3600, 3))

In [7]:
transaction["ACCOUNT_NUMBER"].duplicated().count()

847544

In [8]:
transaction = transaction.drop_duplicates(["ACCOUNT_NUMBER"])
transaction.shape

(5587, 11)

In [9]:
transaction = transaction.iloc[:5144, :]
transaction.head()

,TRANSACTION_DESCRIPTION,CUSTOMER_IDENTIFIER,RECORD_DATE,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE
0,NaN,ID_017ba29a4d371608fad8313ff1d46c8bb42ef830ee0...,2022-07-13 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,1857.56,1000.00,11785,SAVE,internet,89fe98c0e6ccddcf9a0368507f02773693e01dfeb6911c...,NaN
322,POS PURCHASE,ID_0925f20edcf26977988f43469681d58441b449a88c7...,2022-06-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,24.59,-38.91,50,CHEQ,system,0e77f4d244bd3833b4aa54d3ed788a1d99e127b1cf63fd...,c9681cfc2cec371c82a1d38bad5fc9416868c36d17d0df...
329,NaN,ID_09b52b96ac7bc78862e8340d3a7be0fc103227f9bcc...,2022-04-30 00:00:00+02:00,e17913461e7c03cd0f2c1ac41584dcd801cb32e7f86ad9...,2175.76,-33.99,1656,SAVE,system,7d7506943996ff370ac93bf43200a318fbf291013aea9a...,NaN
553,POS PURCHASE,ID_13da6738cf3191e70f74c127b14345e4ddc9bc04adc...,2021-12-09 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,2232.80,-51.50,14761,CHEQ,system,db1d38f2520c0faeb26893765d4184bdd1c94e987ccfea...,2156f36cc7b365fc16661bef94f3a303a3fd4c47bfa242...
904,NaN,ID_1af3755a6e5931439773fcb2ec4dda1190dfd2fece3...,2022-04-27 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,3102.59,-333.24,8005,SAVE,system,630aad4f576667271bf2848309e206465a343c8295480b...,NaN


In [10]:
transaction.shape

(5144, 11)

In [11]:
income.head()

,income_group_code,income_group_desc
0,0,NOT SUPPLIED
1,1,R 0 - 499
2,2,R500 - 999
3,3,"R1,000 - 1,999"
4,4,"R2,000 - 2,999"


In [12]:
income.shape

(22, 2)

In [13]:
test.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,2022-01
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03


## Here we are going to combine train and test set to better proprocess it.

here are few thing to keep in Mind:
    1. Data leakage is something that occurs when we concatinate train and test while separatin them again
    2. We should be careful about the shape of the train and the test on split

## Here I am going to use this method

Add an indicator column while concatenating the two dataframes, so you can later seperate them again:

df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])
Then later you can split them again:

test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]


from this (Link)['https://datascience.stackexchange.com/questions/81617/how-to-combine-and-separate-test-and-train-data-for-data-cleaning']

we are going to concatinate our train and test data

In [14]:
# data = pd.concat([train.assign(ind="train"), test.assign(ind="test")]).reset_index(
#     drop=True
# )

In [15]:
train["type"] = "train"
test["type"] = "test"
data = pd.concat([test, train])

In [16]:
test.shape, train.shape

((1544, 3), (3600, 4))

In [17]:
emp.head()

,employee_status_code,employee_status_desc
0,0,Resigned
1,3,Active
2,1,Inactive
3,2,Retiree


In [18]:
custom.head()

,CUSTOMER_IDENTIFIER,DATE_LAST_UPDATED,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE
0,ID_00a7d65f82a6819e79c29a84e805777f0265308b655...,2022-09,1,6,4,7,51
1,ID_09651edcac6c55a5e401062b0c0ef7bbba3f3e49538...,2022-09,0,7,10,3,32
2,ID_17215674e76eb33b47c4dd3ac3392e3e8f1a34b131b...,2022-09,1,7,7,1,41
3,ID_212860dd16c79ea8eae97cc1fc181e6afd4862fceaf...,2022-09,0,13,10,1,44
4,ID_24953f74da44bd0915c4cb78c1d97cbbd66e85bbd91...,2022-08,0,13,6,1,44


In [19]:
ss.head()

,CUSTOMER_IDENTIFIER,DECLARED_NET_INCOME
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,0
1,ID_c28fb2d93c7b4be50a57aa7ebbe31a6bae33a18e1c6...,0
2,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,0
3,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,0
4,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,0


In [20]:
custom.shape

(5144, 7)

In [21]:
data = data.merge(custom, on="CUSTOMER_IDENTIFIER")
data = data.merge(transaction, on="CUSTOMER_IDENTIFIER")
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,type,DECLARED_NET_INCOME,DATE_LAST_UPDATED,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,TRANSACTION_DESCRIPTION,RECORD_DATE_y,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06,test,NaN,2022-09,1,17,19,1,28,ATM WITHDRAWAL,2022-06-17 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,700.00,-300.00,71,CHEQ,atm,9cd783e94d02ac9b1c90a3f738c404d0c5e7dff06929d6...,f90ddd77e400dfe6a3fcf479b00b1ee29e7015c5bb8cd7...
1,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05,test,NaN,2022-07,0,16,8,1,32,IMDTE DIGITAL PMT,2022-05-30 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,17337.58,-4200.00,129,CHEQ,internet,aadaa1382e79d56d45db9834d35adcaaa7aa8ed6239a5c...,ea71155475a487f7097bfb00598b7ac9fe70b0f49a293f...
2,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05,test,NaN,2022-09,1,2,6,1,45,NaN,2021-11-25 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,19325.41,-913.20,9000,SAVE,system,901f012710fb8e7dcebd60b3cc4a4fba5ce280c4da4e04...,NaN
3,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03,test,NaN,2022-04,0,9,13,1,29,NaN,2021-09-28 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,69.97,-215.27,211,SAVE,system,e30a229eeb3757a881b615a9e95031ce350fb549232629...,NaN
4,ID_782813471fe3e5a8768c24b183f346b295c6f4b2635...,2022-04,test,NaN,2022-07,0,15,8,1,28,POS PURCHASE,2022-04-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,13140.66,-155.00,102,CHEQ,system,b3e4b31d7640722675c5e30042bb1d058379b0547f7b42...,28b8fbc978e5f6c9013eff046fc46d9eb4966213c0db72...


## We are going to fix the data type of the target variable it's categorical we are converting it into int type
Here there is a problem we notice that there is **,** in our target variable I got help from this one: [Convert to Numerical](https://stackoverflow.com/questions/55557004/getting-attributeerror-float-object-has-no-attribute-replace-error-while)

In [22]:
data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(str)
data["DECLARED_NET_INCOME"] = [x.replace(",", "") for x in data["DECLARED_NET_INCOME"]]

In [23]:
data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(float)
# data["DECLARED_NET_INCOME"].apply(lambda x: str(x.replace(",", "").strip()))

In [24]:
data.shape

(5144, 20)

In [25]:
data.dtypes

CUSTOMER_IDENTIFIER          object
RECORD_DATE_x                object
type                         object
DECLARED_NET_INCOME         float64
DATE_LAST_UPDATED            object
SEX_CODE                      int64
INCOME_GROUP_CODE             int64
NUMBER_OF_ACCOUNTS            int64
OCCUPATIONAL_STATUS_CODE      int64
AGE                           int64
TRANSACTION_DESCRIPTION      object
RECORD_DATE_y                object
ACCOUNT_TYPE_CODE            object
ACCOUNT_BALANCE             float64
AMT                         float64
EVENT_NUMBER                  int64
PRODUCT_CODE                 object
CHANNEL                      object
ACCOUNT_NUMBER               object
TRANSACTION_REFERENCE        object
dtype: object

In [26]:
data["RECORD_DATE"] = pd.to_datetime(data["RECORD_DATE_x"])
# data["YEAR"] = data["RECORD_DATE"].dt.year
data["MONTH"] = data["RECORD_DATE"].dt.month
# data["DAY"] = data["RECORD_DATE"].dt.day

In [27]:
# converting the target to numeric

# data["DECLARED_NET_INCOME"] = data["DECLARED_NET_INCOME"].astype(float)

In [28]:
data.drop(["RECORD_DATE", "DATE_LAST_UPDATED"], axis=1, inplace=True)
data.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,type,DECLARED_NET_INCOME,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,TRANSACTION_DESCRIPTION,RECORD_DATE_y,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE,MONTH
0,ID_005e5c1206b2d9857432a381df19989353c3e1b6b0d...,2022-06,test,NaN,1,17,19,1,28,ATM WITHDRAWAL,2022-06-17 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,700.00,-300.00,71,CHEQ,atm,9cd783e94d02ac9b1c90a3f738c404d0c5e7dff06929d6...,f90ddd77e400dfe6a3fcf479b00b1ee29e7015c5bb8cd7...,6
1,ID_94bbc5ce0228e1d25179755dacdf9fc7359a5bbc74a...,2022-05,test,NaN,0,16,8,1,32,IMDTE DIGITAL PMT,2022-05-30 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,17337.58,-4200.00,129,CHEQ,internet,aadaa1382e79d56d45db9834d35adcaaa7aa8ed6239a5c...,ea71155475a487f7097bfb00598b7ac9fe70b0f49a293f...,5
2,ID_41ccd69246cea3c1b4d9349f21d7af2dd092cf7eed4...,2022-05,test,NaN,1,2,6,1,45,NaN,2021-11-25 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,19325.41,-913.20,9000,SAVE,system,901f012710fb8e7dcebd60b3cc4a4fba5ce280c4da4e04...,NaN,5
3,ID_09e29a91e9369a11075b1fd7fd209f3a0c96fb18432...,2022-03,test,NaN,0,9,13,1,29,NaN,2021-09-28 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,69.97,-215.27,211,SAVE,system,e30a229eeb3757a881b615a9e95031ce350fb549232629...,NaN,3
4,ID_782813471fe3e5a8768c24b183f346b295c6f4b2635...,2022-04,test,NaN,0,15,8,1,28,POS PURCHASE,2022-04-26 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,13140.66,-155.00,102,CHEQ,system,b3e4b31d7640722675c5e30042bb1d058379b0547f7b42...,28b8fbc978e5f6c9013eff046fc46d9eb4966213c0db72...,4


In [29]:
data.shape

(5144, 20)

In [31]:
train = data[data["type"] == "train"]

test = data[data["type"] == "test"]

In [32]:
# test, train = data[data["ind"].eq("test")], data[data["ind"].eq("train")]

In [33]:
train.head()

,CUSTOMER_IDENTIFIER,RECORD_DATE_x,type,DECLARED_NET_INCOME,SEX_CODE,INCOME_GROUP_CODE,NUMBER_OF_ACCOUNTS,OCCUPATIONAL_STATUS_CODE,AGE,TRANSACTION_DESCRIPTION,RECORD_DATE_y,ACCOUNT_TYPE_CODE,ACCOUNT_BALANCE,AMT,EVENT_NUMBER,PRODUCT_CODE,CHANNEL,ACCOUNT_NUMBER,TRANSACTION_REFERENCE,MONTH
1542,ID_18018255686b76dbc534d0edf0f7b4e9bb6cc3cde15...,2022-07,train,13500.0,0,15,7,1,45,ATM WITHDRAWAL,2022-07-21 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,35789.84,-5000.0,110,CHEQ,atm,ef3feb4f3a73818fce9e49282f40d9b48e933ff178a2ff...,f90ddd77e400dfe6a3fcf479b00b1ee29e7015c5bb8cd7...,7
1543,ID_782a96eb5b14b4cedc7df05389c6e4751c087026513...,2022-04,train,9000.0,0,15,5,1,32,NaN,2021-10-12 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,5896.51,-375.0,4176,SAVE,system,84b8afc7ae6b57b491d04b801aec9c80ba7ea29a639931...,NaN,4
1544,ID_fbf3035c5b8e7da652269551c78d634a48bca823076...,2022-06,train,8000.0,0,7,7,7,59,NaN,2022-02-09 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,6328.70,-1050.0,4262,SAVE,atm,446a207b9b59fb64500b7526ac76740341fbdf8be9c896...,NaN,6
1545,ID_a0081832892639d933929e24ce46203ea417e17ecbe...,2022-08,train,7000.0,1,9,14,7,57,NaN,2022-08-31 00:00:00+02:00,9317537c2329262b5e19d3e29d23422558e519ca8ff0a8...,4571.16,-23.2,1113,SAVE,system,aacf8b857662976b534a5ec2a88fa0b0f26ab915a48971...,NaN,8
1546,ID_86ee557b2a4506ac40331d40370a40e37e0b95b9f23...,2022-07,train,10400.0,0,12,11,1,29,PREPAID DEBIT,2022-07-30 00:00:00+02:00,e92177da406abc0e0a5bce83e853dbdb4e792af521a0ae...,3334.72,-100.0,11552,CHEQ,atm,e920eb3726431af0e5016f29e81fe23dad1bff6bf1720a...,74d3e443746db127c20da735147d0a0b2efe3f73076195...,7


In [34]:
train.shape, test.shape

((3602, 20), (1542, 20))

In [ ]:
idkjak

In [ ]:
ss.shape

In [ ]:
col2drop = [
    "CUSTOMER_IDENTIFIER",
    "ind",
    "DECLARED_NET_INCOME",
    "RECORD_DATE_y",
    "ACCOUNT_TYPE_CODE",
    "ACCOUNT_NUMBER",
    "TRANSACTION_REFERENCE",
    "TRANSACTION_DESCRIPTION",
    "RECORD_DATE_x",
]

In [ ]:
X = train.drop(col2drop, axis=1)
y = train["DECLARED_NET_INCOME"]

In [ ]:
# test = test.drop(["CUSTOMER_IDENTIFIER", "ind", "DECLARED_NET_INCOME"], axis=1)
test = test.drop(col2drop, axis=1)
test.head()

In [ ]:
X.shape

In [ ]:
X["PRODUCT_CODE_COUNT"] = X["PRODUCT_CODE"].map(
    X["PRODUCT_CODE"].value_counts().to_dict()
)
X["CHANNEL_COUNT"] = X["CHANNEL"].map(X["CHANNEL"].value_counts().to_dict())

test["PRODUCT_CODE_COUNT"] = test["PRODUCT_CODE"].map(
    test["PRODUCT_CODE"].value_counts().to_dict()
)
test["CHANNEL_COUNT"] = test["CHANNEL"].map(test["CHANNEL"].value_counts().to_dict())

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

X["PRODUCT_CODE"] = LabelEncoder().fit_transform(X["PRODUCT_CODE"])
X["CHANNEL"] = LabelEncoder().fit_transform(X["CHANNEL"])

In [ ]:
test["PRODUCT_CODE"] = LabelEncoder().fit_transform(test["PRODUCT_CODE"])
test["CHANNEL"] = LabelEncoder().fit_transform(test["CHANNEL"])

In [ ]:
ss.shape

In [ ]:
# test = test.iloc[:1544]
# test.head()

In [ ]:
X.shape, test.shape

In [ ]:
test.dtypes

In [ ]:
y.dtypes

In [ ]:
params = {
    "objective": "quantile",
    "metric": "rmse",
    "learning_rate": 0.02,
    "sub_feature": 0.8,
    "sub_row": 0.75,
    "bagging_freq": 1,
    "lambda_l2": 0.6,
    "verbosity": 1,
    "num_iterations": 5000,
    "num_leaves": 128,
    "min_data_in_leaf": 50,
}

In [ ]:
%%time
errcb = []
y_pred_totcb = []
fold = KFold(n_splits=10, shuffle=True, random_state=42)
i = 1
for train_index, test_index in fold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # m = LGBMRegressor(**params)
    m = CatBoostRegressor()
    # m = XGBRegressor()
    m.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100,
        early_stopping_rounds=200,
    )
    preds = m.predict(X_test)
    print("err: ", metrics.mean_squared_error(y_test, preds, squared=False))
    errcb.append(metrics.mean_squared_error(y_test, preds, squared=False))
    p = m.predict(test)
    y_pred_totcb.append(p)

In [ ]:
np.mean(np.log1p(errcb))

In [ ]:
fea_imp = pd.DataFrame({"imp": m.feature_importances_, "col": X.columns})
fea_imp = fea_imp.sort_values(["imp", "col"], ascending=False).iloc[-30:]
_ = fea_imp.plot(kind="barh", x="col", y="imp", figsize=(20, 10))
plt.savefig("catboost_feature_importance.png")

In [ ]:
np.mean(y_pred_totcb, 0)

In [ ]:
len(np.mean(y_pred_totcb, 0))

In [ ]:
test["DECLARED_NET_INCOME"] = np.mean(y_pred_totcb, 0)

In [ ]:
ss.head()

In [ ]:
ss["DECLARED_NET_INCOME"] = np.mean(y_pred_totcb, 0)
ss.head()

### transforming the prediction


In [ ]:
ss["DECLARED_NET_INCOME"] = np.log1p(abs(ss["DECLARED_NET_INCOME"]))

In [ ]:
ss.head()

In [ ]:
ss.to_csv("base_lgb.csv", index=False)